In [18]:
from urllib.request import urlopen
from urllib import parse
from bs4 import BeautifulSoup
from Models.DataCollector import DataCollector
import torch.nn.functional as F
import json
import operator

In [2]:
with open('./metadata/name_to_key.json', 'r') as fp:
    name_to_key = json.load(fp)

In [3]:
class UserInspector(DataCollector):
    def __init__(self):
        self.name_to_key = {}
        self.name_to_id = {}
        self.id_to_key = {}
        self.key_to_id = {}
        with open('./metadata/name_to_key.json', 'r') as fp:
            self.name_to_key = json.load(fp)

        with open('./metadata/name_to_id.json', 'r') as fp:
            self.name_to_id = json.load(fp)

        with open('./metadata/id_to_key.json', 'r') as fp:
            self.id_to_key = json.load(fp)

        with open('./metadata/key_to_id.json', 'r') as fp:
            self.key_to_id = json.load(fp)

    def user_history_collector(self, userName):
        # url encoding (for Korean words)
        userName = parse.quote(userName)

        # open url and create bs4 Object
        html = urlopen("https://www.op.gg/summoner/userName="+userName)
        bsObject = BeautifulSoup(html, "html.parser")

        # decode userName from url encoding
        userName = parse.unquote(userName)

        # win/loss and play count
        wins = int(bsObject.find('span', 'wins').text[:-1])
        losses = int(bsObject.find('span', 'losses').text[:-1])
        total_play = wins + losses    
        win_rate = round(wins/total_play, 3)

        champ_list = bsObject.find_all("div", {'class' : 'ChampionBox Ranked'})
        champion_history = self.champ_history_setter(champ_list)            

        # this is (adapted) user_history DTO
        user_historyDTO = {
            'user_name' : userName,
            'total_play' : total_play,
            'win_rate' : win_rate,
            'champion_history' : champion_history,
        }
        return user_historyDTO
    
    def user_allchamp_collector(self, userName):
        # url encoding (for Korean words)
        userName = parse.quote(userName)

        # open url and create bs4 Object
        html = urlopen("https://www.op.gg/summoner/champions/userName="+userName)
        bsObject = BeautifulSoup(html, "html.parser")

        # decode userName from url encoding
        userName = parse.unquote(userName)

        champ_list = bsObject.find_all("tr", {'class' : 'Row'})

        champion_history_dict = dict()

        for champion in champ_list:
            champion_name = champion.find('td', 'ChampionName Cell')

            win, lose = 0, 0        
            if champion_name is not None:
                champion_name = champion_name.find('a').text
                champion_win = champion.find('div', 'Text Left')
                champion_lose = champion.find('div', 'Text Right')
                if champion_win is not None:
                    win = int(champion_win.text[:-1])
                if champion_lose is not None:
                    lose = int(champion_lose.text[:-1])

                playcount = win + lose
                winrate = round(win/playcount, 3)
                champion_id = self.name_to_id[champion_name]
                if playcount >= 5:
                    champion_history_dict[champion_id] = [playcount, winrate]

        return champion_history_dict

In [35]:
test = UserInspector()
test.user_allchamp_collector('구마유시')

{'Kaisa': [130, 0.562],
 'Lucian': [104, 0.471],
 'Kalista': [95, 0.453],
 'Draven': [81, 0.531],
 'Vayne': [76, 0.513],
 'Ezreal': [58, 0.552],
 'Sivir': [34, 0.588],
 'Kayn': [26, 0.538],
 'Jhin': [22, 0.591],
 'Neeko': [18, 0.5],
 'Caitlyn': [13, 0.385],
 'Tristana': [13, 0.385],
 'Varus': [12, 0.333],
 'Xayah': [10, 0.6],
 'Yasuo': [10, 0.4],
 'Vladimir': [9, 0.667],
 'Twitch': [7, 0.571],
 'Sion': [7, 0.0],
 'Aatrox': [6, 0.333],
 'Jinx': [5, 0.4]}

In [5]:
import torch

from Models.Models import AutoEncoder, Predictor
from utils import champ_id_remap, global_win_rate, get_original_champ_id

In [67]:
class ChampionRecommender():
    def __init__(self):
        self.user_inspector = UserInspector()

        self.user_encoder = AutoEncoder(143, 12)
        self.user_encoder.load_state_dict(torch.load('./trained_model/user_encoder_augmented.pth'))

        self.item_encoder = AutoEncoder(143, 8)
        self.item_encoder.load_state_dict(torch.load('./trained_model/item_encoder2.pth'))

        self.predictor = Predictor(user_len=12, item_len=8, hidden_unit=10).eval()
        self.predictor.load_state_dict(torch.load('./trained_model/predictor_last_m.pth'))
        
        self.remapped_champ_id = champ_id_remap()
        self.global_win_rate = global_win_rate() 
        
        with open('./datasets/item_vectors_tf_idf.json', 'r') as fp:
            self.item_vectors = json.load(fp)        
        
        with open('./metadata/user_idf_table.json', 'r') as fp:
            self.idf_table = json.load(fp)

    def recommender(self, userName):
        #try:
        user_data = self.user_inspector.user_history_collector(userName)
        win_rate_dict, played_champion_key = self.winrate_predictior(user_data)
        win_rate_dict = self._dict_origin_mapper(win_rate_dict)
        sorted_result = sorted(win_rate_dict.items(), key=operator.itemgetter(1), reverse=True)
        win_rate_dict = dict()
        for item in sorted_result:
            win_rate_dict[item[0]] = item[1]
            
        return win_rate_dict, played_champion_key
        #except:
        #    print("not a valid user. please check.")


    def winrate_predictior(self, user_data):
        user_winrate = self._tensor_item(user_data['win_rate'])
        user_vec = self._user_vector_generator(user_data)
        played_champions = [i for i, e in enumerate(user_vec) if e != 0]
        
        played_champion_key = [get_original_champ_id(self.remapped_champ_id, int(i)) for i in played_champions]
        
        user_vec = torch.Tensor(user_vec)
        user_vec = self.user_encoder.encoder(user_vec)
        
        win_rate_dict = dict()
        
        for key, item in self.item_vectors.items():
            if int(key) not in played_champions:
                global_win = self._tensor_item(self.global_win_rate[int(key)])
                item_vec = torch.Tensor([item])
                item_vec = self.item_encoder.encoder(item_vec)
                #item_vec = torch.Tensor([0,0,0,0,0,0,0,0])
                #user_vec = torch.Tensor([0,0,0,0,0,0,0,0,0,0,0,0])
                item_vec = item_vec.squeeze()
                
                user_vec = F.normalize(user_vec, dim=0)
                item_vec = F.normalize(item_vec, dim=0)
                
                prediction = self.predictor(user_vec, item_vec, user_winrate, global_win)
                win_rate_dict[key] = prediction.item() - global_win.item()
                
        return win_rate_dict, played_champion_key
    
    
    def system_tester(self, userName):
        allchamp = self.user_inspector.user_allchamp_collector(userName)
        recommendation, played_keys = self.recommender(userName)
        
        type1 = []
        type2 = []
        
        for i, (key, item) in enumerate(recommendation.items()):
            if key in allchamp.keys():
                type1.append([i, allchamp[key][1]])
                type2.append(allchamp[key][1])
        
        reco = []
        for i in range(5):
            key = list(recommendation.keys())[i]
            if key in allchamp.keys():
                reco.append(allchamp[key][1])
        
        if len(reco) != 0 and len(type2) != 0:
            type2 = sum(reco)/len(reco) - sum(type2)/len(type2)
        
        return type1, type2
        
    
    def _dict_origin_mapper(self, recommend_dict):
        original_recommend_dict = dict()
        for remapped_id, score in recommend_dict.items():
            original_id = get_original_champ_id(self.remapped_champ_id, int(remapped_id))
            original_name = self.user_inspector.key_to_id[str(original_id)]
            original_recommend_dict[original_name] = recommend_dict[remapped_id]
        return original_recommend_dict

            
    def _user_vector_generator(self, user_data):
        user_win_rate = user_data['win_rate']
        play_count_vector = [0 for x in range(143)]

        for champ_history in user_data['champion_history']:
            original_champ_id = champ_history['champion_key']
            champ_idx = self.remapped_champ_id[original_champ_id]

            max_play_count = self._get_max_play_count(user_data)
            measured_score = (champ_history['play_count'] / max_play_count) * self.idf_table[str(champ_idx)]
            play_count_vector[champ_idx] = measured_score
        
        return play_count_vector

    
    def _get_max_play_count(self, user_data):
        """
        Get count of maximally played champion for user_name
        """
        max_play_count = 0

        for champ_history in user_data['champion_history']:
            champ_play_count = champ_history['play_count']
            if max_play_count < champ_play_count:
                max_play_count = champ_play_count

        return max_play_count

    
    def _tensor_item(self, item):
        return torch.Tensor([item])

In [68]:
test = ChampionRecommender()

In [69]:
#test_output, keys = test.recommender('이문용')

In [78]:
with open('./data_batch/userlist1.json', 'r') as fp:
    userlist = json.load(fp)

type1 = []
type2 = []
    
for i, key in enumerate(userlist['user_name']):
    #try:
    if i < 3000:
        type1_elem, type2_elem = test.system_tester(key)
        type1 += type1_elem
        type2.append(type2_elem)
    #except:
    #    pass

KeyError: 'Yuumi'

In [73]:
type1

[]